In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import  VotingClassifier
from sklearn.model_selection import  cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import  r2_score
from sklearn.metrics import  mean_squared_error
from sklearn.model_selection import cross_val_score

In [40]:
def convet(df):
    #将类别型特征转换为数值型特征
    train_data=df.copy()
    for col in train_data.columns: #遍历每一个特征
        if train_data[col].dtype=='O':  
            n=1  #用该数值来替换类别特征值
            for value in set(train_data[col]):
                train_data.ix[train_data[col]==value,col]=n
                n+=1
    return(train_data)

In [41]:
#读取数据
df_train=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [42]:
#准备训练数据
index=list(df_train.columns)
train_data=df_train[index[:-1]]
train_label=df_train[index[-1]]
#准备测试数据
#index=list(test_data.columns)
#test_data = test_data [index[:-1]] 
#test_label = test_data [index[-1]]

In [43]:
#去除特征值缺失比例超出50%的特征
NAs=pd.concat([train_data.isnull().sum(),test_data.isnull().sum()],axis=1,keys=['train','test'])
#从训练集中筛选出特征值缺失大于50%的特征
feature1 =train_data.columns[train_data.isnull().sum()/train_data.shape[0]>0.5]
#从测试集中筛选出特征值缺失大于50%的特征
feature2 =test_data.columns[test_data.isnull().sum()/test_data.shape[0]>0.5]
#合并所有需要去除的特征名称
features_unioned=set.union(set(feature1),set(feature2),set(['Utilities', 'RoofMatl', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'LowQualFinSF',
               'BsmtFullBath', 'BsmtHalfBath', 'Functional', 'GarageYrBlt', 'GarageArea', 'GarageCond', 'WoodDeckSF',
               'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal']))
print(feature1)
print(feature2)
print(features_unioned)
#去除特征
train_data=train_data[train_data.columns.difference(list(features_unioned))]
test_data=test_data[test_data.columns.difference(list(features_unioned))]

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')
Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')
{'Heating', 'ScreenPorch', 'EnclosedPorch', '3SsnPorch', 'Alley', 'Utilities', 'BsmtFinSF2', 'PoolArea', 'GarageCond', 'MiscVal', 'GarageYrBlt', 'BsmtUnfSF', 'WoodDeckSF', 'BsmtFullBath', 'RoofMatl', 'PoolQC', 'BsmtHalfBath', 'GarageArea', 'BsmtFinSF1', 'MasVnrArea', 'OpenPorchSF', 'FireplaceQu', 'LowQualFinSF', 'Fence', 'MiscFeature', 'Functional'}


填充缺失值

In [44]:

features=train_data.copy()
features['MSSubClass'] = features['MSSubClass'].astype(str)

# 填充最频繁的特征值
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

# 填充平均值
features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())


features.OverallCond = features.OverallCond.astype(str)

# 填充最频繁的特征值
features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
# 缺失值指代为NoBasement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('NoBSMT')

# NA 指代为 0
#features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

# 填充最频繁的特征值
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

# 填充最频繁的特征值
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

# 缺失值指代为NoFP
#features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

#缺失值指代为No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    features[col] = features[col].fillna('NoGRG')

# 缺失值指代为0
features['GarageCars'] = features['GarageCars'].fillna(0.0)

# 填充最频繁的特征值
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])


features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
train_data=features

填充缺失值

In [45]:

features=test_data.copy()
features['MSSubClass'] = features['MSSubClass'].astype(str)

# 填充最频繁的特征值
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

# 填充平均值
features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())


features.OverallCond = features.OverallCond.astype(str)

# 填充最频繁的特征值
features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
# 缺失值指代为NoBasement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('NoBSMT')

# NA 指代为 0
#features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

# 填充最频繁的特征值
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

# 填充最频繁的特征值
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

# 缺失值指代为NoFP
#features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

#缺失值指代为No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    features[col] = features[col].fillna('NoGRG')

# 缺失值指代为0
features['GarageCars'] = features['GarageCars'].fillna(0.0)

# 填充最频繁的特征值
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])


features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
test_data=features

In [46]:
#log平滑
train_label=np.log(train_label)
#归一化
for col in ['LotFrontage', 'LotArea', 'GrLivArea', 'TotalSF']:
    train_data[col]=(train_data[col]-train_data[col].mean())/train_data[col].std()

将类别型特征值转换为数值型

In [47]:
train_data=convet(df=train_data)
test_data=convet(df=test_data)

In [71]:
test_data.max()

BsmtFinType1         7.0
HalfBath             2.0
TotRmsAbvGrd        15.0
Fireplaces           4.0
Street               2.0
LotArea          56600.0
KitchenQual          4.0
YearBuilt         2010.0
LandSlope            3.0
MSSubClass          16.0
ExterQual            4.0
PavedDrive           3.0
CentralAir           2.0
GarageFinish         4.0
BsmtCond             5.0
SaleCondition        6.0
GrLivArea         5095.0
Condition2           5.0
TotalSF          10190.0
FullBath             4.0
HeatingQC            5.0
OverallQual         10.0
Foundation           6.0
GarageCars           5.0
BsmtQual             5.0
GarageType           7.0
KitchenAbvGr         3.0
YearRemodAdd      2010.0
Neighborhood        25.0
dtype: float64

特征选择

In [48]:
#基于模型的特征排序
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.cross_validation import  KFold
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=20, max_depth=4,n_jobs=-1)
X=np.array(train_data.values)
Y=np.array(train_label.values)
names=train_data.columns
scores = []
for i in range(X.shape[1]):
    score = cross_val_score(rf, X[:, i:i+1], Y, scoring="r2",
                              cv=ShuffleSplit(len(X), 3, .3))
    scores.append((round(np.mean(score), 3), names[i]))
print(sorted(scores, reverse=True))

[(0.67900000000000005, 'OverallQual'), (0.65700000000000003, 'TotalSF'), (0.54500000000000004, 'GrLivArea'), (0.46899999999999997, 'GarageCars'), (0.45500000000000002, 'KitchenQual'), (0.45400000000000001, 'YearBuilt'), (0.441, 'ExterQual'), (0.441, 'BsmtQual'), (0.38800000000000001, 'Neighborhood'), (0.38400000000000001, 'GarageFinish'), (0.36199999999999999, 'FullBath'), (0.33100000000000002, 'GarageType'), (0.29599999999999999, 'Foundation'), (0.29099999999999998, 'YearRemodAdd'), (0.27400000000000002, 'TotRmsAbvGrd'), (0.27100000000000002, 'MSSubClass'), (0.255, 'Fireplaces'), (0.22500000000000001, 'LotArea'), (0.221, 'HeatingQC'), (0.215, 'BsmtFinType1'), (0.19400000000000001, 'MasVnrType'), (0.16500000000000001, 'LotFrontage'), (0.152, 'GarageQual'), (0.14399999999999999, 'MSZoning'), (0.14199999999999999, 'Exterior2nd'), (0.13, 'OverallCond'), (0.129, 'BsmtExposure'), (0.123, 'SaleCondition'), (0.112, 'SaleType'), (0.10100000000000001, 'PavedDrive'), (0.10000000000000001, 'Centr

In [49]:
#基于递归特征消除（RFE）
from sklearn.feature_selection import  RFE
from sklearn.linear_model import  Ridge
lr_ridge=Ridge(alpha=0.15)
rfe=RFE(lr_ridge,n_features_to_select=1)
rfe.fit(train_data,train_label)
print(sorted(zip(rfe.ranking_,train_data.columns),reverse=False))

[(1, 'CentralAir'), (2, 'TotalSF'), (3, 'OverallQual'), (4, 'GarageCars'), (5, 'Street'), (6, 'FullBath'), (7, 'Fireplaces'), (8, 'KitchenAbvGr'), (9, 'PavedDrive'), (10, 'LandSlope'), (11, 'HalfBath'), (12, 'KitchenQual'), (13, 'Condition2'), (14, 'LotArea'), (15, 'ExterQual'), (16, 'BsmtQual'), (17, 'BsmtCond'), (18, 'SaleCondition'), (19, 'TotRmsAbvGrd'), (20, 'GarageType'), (21, 'ExterCond'), (22, 'LotConfig'), (23, 'LandContour'), (24, 'GrLivArea'), (25, 'BsmtExposure'), (26, 'HeatingQC'), (27, 'LotShape'), (28, 'Condition1'), (29, 'BsmtFinType2'), (30, 'RoofStyle'), (31, 'MasVnrType'), (32, 'GarageQual'), (33, 'BldgType'), (34, 'HouseStyle'), (35, 'MSSubClass'), (36, 'Electrical'), (37, 'Foundation'), (38, 'YrSold'), (39, 'SaleType'), (40, 'BedroomAbvGr'), (41, 'YearRemodAdd'), (42, 'Exterior1st'), (43, 'GarageFinish'), (44, 'MSZoning'), (45, 'OverallCond'), (46, 'BsmtFinType1'), (47, 'MoSold'), (48, 'YearBuilt'), (49, 'Exterior2nd'), (50, 'LotFrontage'), (51, 'Neighborhood'), (5

In [50]:
#各自去Top20的特征，合并为最终训练模型的特征子集
feature_subset=[col_name for _, col_name in sorted(scores, reverse=True)[:20]]
feature_=train_data.columns[rfe.ranking_<=20]
feature_subset.extend(feature_)
feature_subset=set(feature_subset)

In [51]:
train_data , test_data = train_data[list(feature_subset)] , test_data[list(feature_subset)]

In [55]:
train_data,valid_data = train_data[:1200] , train_data[1200:]
train_label,valid_label = train_label[:1200] , train_label[1200:]

In [56]:
#并行调参
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
clf_RF=RandomForestRegressor(n_jobs=-1)
parameters={'criterion':('mae','mse'),'max_features':(10,15,20,25),'max_depth':(3,4,5,6)}
grid_search=GridSearchCV(clf_RF,param_grid=parameters,verbose=1)


In [57]:
grid_search.fit(train_data,train_label)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:   25.8s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'criterion': ('mae', 'mse'), 'max_features': (10, 15, 20, 25), 'max_depth': (3, 4, 5, 6)},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [58]:
print(grid_search.best_params_)
print(grid_search.best_score_)
best_estimator=grid_search.best_estimator_
best_estimator.fit(train_data,train_label)
best_estimator.score(valid_data,valid_label)

{'criterion': 'mse', 'max_features': 15, 'max_depth': 6}
0.8548687306527495


0.83652344336001327

In [72]:
#利用训练好的模型预测
predicted_result_1=best_estimator.predict(test_data)

In [59]:
#build the AdaBoost
from sklearn.ensemble import  AdaBoostRegressor
from sklearn.tree import  DecisionTreeRegressor
from sklearn.grid_search import  GridSearchCV

regr=DecisionTreeRegressor(random_state=33)
clf_Ada=AdaBoostRegressor(base_estimator=regr,n_estimators=100,random_state=44)
pipeline=Pipeline([('regr',regr),('clf_Ada',clf_Ada)])
parameters={'regr__criterion':('mse','mae'),'regr__splitter':('random','best'),'regr__min_samples_split':(2,3,4),
           'clf_Ada__learning_rate':(0.1,0.2)}
grid_search=GridSearchCV(estimator=pipeline,param_grid=parameters)
grid_search.fit(train_data,train_label)

/Users/HankPeng/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/HankPeng/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/HankPeng/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)
/Users/HankPeng/anaconda/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarnin

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('regr', DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=33,
           splitter='best')...er='best'),
         learning_rate=1.0, loss='linear', n_estimators=100,
         random_state=44))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'regr__criterion': ('mse', 'mae'), 'regr__splitter': ('random', 'best'), 'regr__min_samples_split': (2, 3, 4), 'clf_Ada__learning_rate': (0.1, 0.2)},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [73]:
print(grid_search.best_score_)
print(grid_search.best_params_)
best_estimator=grid_search.best_estimator_

0.837967016714252
{'clf_Ada__learning_rate': 0.1, 'regr__criterion': 'mae', 'regr__min_samples_split': 2, 'regr__splitter': 'best'}


In [74]:
best_regr=DecisionTreeRegressor(splitter='random',min_samples_split=3,criterion='mae',random_state=33)
best_Ada=AdaBoostRegressor(base_estimator=best_regr,learning_rate=0.2)
best_Ada.fit(train_data,train_label)
best_Ada.score(valid_data,valid_label)

0.84979027487012893

In [76]:
#利用训练好的模型预测
predicted_result_2=best_estimator.predict(test_data)